In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import json
import ta
import matplotlib.pyplot as pl
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin

#NVR	332.758744
#BKNG	220.505001
#AZO	110.754999
#MTD	81.909869

In [3]:
all_stocks = ['NVR', 'BKNG', 'AZO', 'MTD']

In [4]:
start_date = '2014-01-01'
end_date = '2024-01-01'

# Download the data
df = yf.download(all_stocks, start=start_date, end=end_date)['Adj Close']
df = df.reset_index()
# Save the data to a CSV file
df.to_csv('stocks_data.csv')

# Display the 1st 5 records
df.head(5)

[*********************100%%**********************]  4 of 4 completed


Ticker,Date,AZO,BKNG,MTD,NVR
0,2014-01-02,474.109985,1139.892334,241.529999,1017.099976
1,2014-01-03,475.500000,1127.194214,242.710007,1019.349976
2,2014-01-06,478.769989,1134.010986,242.419998,1025.630005
3,2014-01-07,483.589996,1157.436890,248.360001,1013.880005
4,2014-01-08,489.070007,1151.893921,249.669998,1009.900024


In [4]:
df

Ticker,Date,AZO,BKNG,CMG,NVR
0,2013-01-02,356.880005,635.308105,6.021200,930.130005
1,2013-01-03,356.140015,645.588013,6.019000,949.000000
2,2013-01-04,360.850006,645.269592,6.003600,954.489990
3,2013-01-07,356.170013,654.196167,5.991800,966.890015
4,2013-01-08,348.250000,654.235962,5.955200,974.500000
...,...,...,...,...,...
2574,2023-03-24,2329.399902,2487.225098,32.485001,5413.000000
2575,2023-03-27,2368.550049,2496.161865,32.956402,5375.450195
2576,2023-03-28,2405.219971,2532.793701,33.057999,5413.560059
2577,2023-03-29,2395.370117,2559.314453,33.134399,5433.990234


In [5]:
# plot the 4 stocks together 


df1 = pd.melt(df, id_vars='Date')
px.line(df1, x = 'Date', y = 'value', color='Ticker')

**The 4 stocks have a diverse range. We would take each stock one at a time**

In [6]:
df

Ticker,Date,AZO,BKNG,CMG,NVR
0,2013-01-02,356.880005,635.308105,6.021200,930.130005
1,2013-01-03,356.140015,645.588013,6.019000,949.000000
2,2013-01-04,360.850006,645.269592,6.003600,954.489990
3,2013-01-07,356.170013,654.196167,5.991800,966.890015
4,2013-01-08,348.250000,654.235962,5.955200,974.500000
...,...,...,...,...,...
2574,2023-03-24,2329.399902,2487.225098,32.485001,5413.000000
2575,2023-03-27,2368.550049,2496.161865,32.956402,5375.450195
2576,2023-03-28,2405.219971,2532.793701,33.057999,5413.560059
2577,2023-03-29,2395.370117,2559.314453,33.134399,5433.990234


**Install Optuna, Jupyterlab-Optuna, Optuna-Dashboard**

An open source hyperparameter optimization framework to automate hyperparameter search



In [11]:
df.head(2)

Ticker,Date,AZO,BKNG,CMG,NVR
0,2013-01-02,356.880005,635.308105,6.0212,930.130005
1,2013-01-03,356.140015,645.588013,6.0190,949.000000


In [12]:
# Normalize the data
data = df[['AZO']]

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create the time series sequences
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

seq_length = 60  # e.g., use 60 days of data to predict the next day
X, y = create_sequences(scaled_data, seq_length)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [14]:
len(X)

2519

In [15]:
len(X[0])

60

**Define the LSTM Model**

In [17]:
def create_model(units=50, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(seq_length, 1)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

**Built Custom Wrapper**

In [22]:

class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, units=50, dropout_rate=0.2, epochs=10, batch_size=32):
        self.units = units
        self.dropout_rate = dropout_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None

    def fit(self, X, y):
        self.model = create_model(units=self.units, dropout_rate=self.dropout_rate)
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        return -self.model.evaluate(X, y, verbose=0)


**HyperParameter Tuning**

https://keras.io/api/layers/recurrent_layers/lstm/

https://keras.io/guides/keras_tuner/getting_started/

**Grid Search**

In [23]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'units': [50, 100],
    'dropout_rate': [0.2, 0.5],
    'batch_size': [32, 64],
    'epochs': [10, 20]
}

model = KerasRegressorWrapper()

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

c:\Users\SHEKHAR\.conda\envs\aws\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


c:\Users\SHEKHAR\.conda\envs\aws\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


c:\Users\SHEKHAR\.conda\envs\aws\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



...

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


c:\Users\SHEKHAR\.conda\envs\aws\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step


c:\Users\SHEKHAR\.conda\envs\aws\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Best: -0.0001246015718771528 using {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 20, 'units': 100}


**Grid Search**

Best: -0.0001246015718771528 using {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 20, 'units': 100}